# Flyweight

> Optimizing memory usage

Let's suppose we are building a multiplayer game. We have a `User` class that stores the full names of our users.

In [1]:
class User:
    def __init__(self, name):
        self.name = name

Since many people have similar or even identical names, if we simply store the full name of each user, we will end up with lots of duplicate data.

Let's simulate this scenario by generating random names for a few users.

In [3]:
import random
import string

def random_string():
    chars = string.ascii_lowercase
    return ''.join([random.choice(chars) for x in range(8)])

users = []

first_names = [random_string() for x in range(100)]
last_names = [random_string() for x in range(100)]

# let's make 10,000 Users!
for first in first_names:
    for last in last_names:
        users.append(User(f'{first} {last}'))
        
print(len(users))

10000


In this example we've ended up with 10,000 unique users, even though we are only have 100 first names and 100 last names. We are consuming memory for 10k strings when we could be using only the memory for 200 strings instead.

We can optimize our `User` class with the **flyweight pattern**: we will build a class which is simply a pointer into some common store.

Our new `User2` class will contain a static list of all the common first and last names. When we initialize a new user with a particular name (which for simplicity's sake we will assume it will always be a first and a last name separated with a space), we will store both first and last name in the static list (if they weren't present before) and the user instance will contain indices of the position of the first and last name in the static list instead.

In [4]:
class User2:
    strings = [] # static list / store

    def __init__(self, full_name):
        def get_or_add(s):
            """Get the index of name s or add it to the store and return its index"""
            if s in self.strings:
                return self.strings.index(s)
            else:
                self.strings.append(s)
                return len(self.strings)-1
        self.names = [get_or_add(x) for x in full_name.split(' ')] # indices for the fist and last name of the user

    def __str__(self):
        return ' '.join([self.strings[x] for x in self.names])

(We also make our new class printable by making it string representable with `__str__`).

Let's test our new class:

In [5]:
u2 = User2('Jim Jones')
u3 = User2('Frank Jones')
print(u2.names)
print(u3.names)
print(User2.strings)

users2 = []

first_names = [random_string() for x in range(100)]
last_names = [random_string() for x in range(100)]

for first in first_names:
    for last in last_names:
        users2.append(User2(f'{first} {last}'))
        
print(len(users2))
print(len(User2.strings))

[0, 1]
[2, 1]
['Jim', 'Jones', 'Frank']
10000
203


As you can see above, even though we've also created 10k `User2` instances, we are only storing 203 names in total (the 3 we create in the first lines and the 200 we randomly generate afterwards). Each instance of `User2` only stores the indices to the names.